[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/Pesquisa-Operacional-III-A/blob/main/9_PNL.ipynb)

## **Pesquisa Operacional III-A**

**Professores:**
- Diogo Ferreira de Lima Silva (TEP-UFF)
- Marcos Costa Roboredo (TEP-UFF)


# Programação Não Linear

Para resolver os problemas de PNL em uma ferramenta computacional, precisamos de uma linguagem de modelagem matemática e um solver. 

Vimos anteriormente alguns exemplos de modelagem e Pyomo com posterior uso do solver Gurobi. 

A boa notícia é que podemos continuar usando o mesmo framework de modelagem. A única diferença é que precisaremos de um resolvedor diferente.

## Instalação das bibliotecas

Inicialmente, se estiver trabalhando no *Google Colab*, você precisará instalar a biblioteca Pyomo. 

Neste caso, execute o bloco de código abaixo.

In [ ]:
!pip install pyomo # Instala o Pyomo no ambiente de execução

Em seguida, para a instalação do *solver*, execute as duas células abaixo:

In [ ]:
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"

In [ ]:
!unzip -o -q ipopt-linux64

## Modelando um problema de PNL

Precisamos importar a biblioteca:

In [ ]:
import pyomo.environ as pe 

Modelo

In [ ]:
# MODELO
M = pe.ConcreteModel() # Criando o modelo concreto para o nosso problema

# VARIÁVEIS
x = M.x = pe.Var(domain= pe.PositiveReals) # Definindo a variável x
y = M.y = pe.Var(domain= pe.PositiveReals) # Definindo a variável y

# FUNÇÃO OBJETIVO
z = M.z = pe.Objective(expr= -3 * x**2 + y, sense= pe.maximize) # Definindo a função objetivo

# RESTRIÇÃO
M.R1 = pe.Constraint(expr= x**2 + y**2 <= 1) # Definindo uma restrição de igualdade


# SOLUCIONANDO O MODELO
pe.SolverFactory('ipopt').solve(M)

# Imprime o resultado
print(f"z= {pe.value(M.z)}\nx= {pe.value(M.x)}\ny= {pe.value(M.y)}")